In [1]:
!nvidia-smi

Fri Apr 15 06:35:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%cd /content/drive/MyDrive/4-1_Capstone

/content/drive/MyDrive/4-1_Capstone


In [3]:
import torch
import os
import numpy as np
import pandas as pd

In [4]:
train_y = "./datasets/multi_train_y.csv"
val_y   = "./datasets/multi_val_y.csv"
# train_y = "/content/train_y.csv"
# val_y   = "/content/val_y.csv"

train_y = pd.read_csv(train_y)
val_y   = pd.read_csv(val_y)

In [5]:
train_img_paths = "src_datasets/train/images"
val_img_paths   = "src_datasets/validation/images"

from natsort import natsorted
from glob import glob


train_imgs = natsorted(glob(os.path.join(train_img_paths, '*')))
val_imgs   = natsorted(glob(os.path.join(val_img_paths, '*')))

In [6]:
# import torch
# import torch.nn

# from torch.nn import Conv2d, Linear, MaxPool2d, ReLU
# import torch.nn as nn

# class Model(nn.Module):
#     def __init__(self):
#         super().__init__()

#                     # i_c = 224, 224                    
#         self.conv1 = Conv2d(3  , 256, 3, 1, 1) # i_c, o_c, k, s, p
#         self.pool1 = MaxPool2d(2, 2)
#         self.activation1 = ReLU()
#         self.conv2 = Conv2d(256, 256, 3, 1, 1)
#         self.pool2 = MaxPool2d(2, 2)
#         self.activation2 = ReLU()
#         self.linear1 = Linear(56 * 56 * 256, 7) # nb_classes

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.pool1(x)
#         x = self.activation1(x)
#         x = self.conv2(x)
#         x = self.pool2(x)
#         x = self.activation2(x)
#         x = x.view(x.size(0), -1)
#         x = self.linear1(x)
        
#         return x
                

In [7]:
from torch.utils.data import Dataset

from PIL import Image
class MyDataset(Dataset):

    def __init__(self, transform=None, split=None):        
        
        self.split = split

        if split.upper() == 'TRAIN':
            self.img_dirs = train_imgs
            self.y = train_y.values
            self.transform = transform
        
        if split.upper() == 'VALIDATION':
            self.img_dirs = val_imgs
            self.y = val_y.values
            self.transform = transform

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):        
        image = Image.open(self.img_dirs[idx])
        image = image.convert('RGB')
        
        y = self.y[idx]

        if self.transform:
            image = self.transform(image)

        return image, y
    

from torchvision import transforms
tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])

In [8]:
!pip install timm -q
import torch
from torch.nn import Linear
import timm
timm.list_models()
model = timm.create_model(model_name = 'efficientnet_b0', pretrained=True)
model.classifier = Linear(in_features=1280, out_features=7, bias=True)
# model

     |████████████████████████████████| 431 kB 12.8 MB/s 


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth


In [9]:
nb_epochs = 3
lr = 1e-2
batch_size = 32
# batch_size = 64
n_classes = 7

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# model = Model().to(device)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True)
# criterion = torch.nn.CrossEntropyLoss().to(device)
criterion = torch.nn.BCEWithLogitsLoss().to(device)

from torch.utils.data import TensorDataset, DataLoader

train_loader = DataLoader(MyDataset(tf, split='TRAIN'),
                        batch_size=batch_size,
                        shuffle=True,
                        drop_last=True,
                        pin_memory=True,
                        num_workers=4)


validation_loader = DataLoader(MyDataset(tf, split='VALIDATION'),
                        batch_size=batch_size,
                        shuffle=False,
                        drop_last=False,
                        pin_memory=True,
                        num_workers=4)


from tqdm import tqdm
for epoch in tqdm(range(nb_epochs)):
    print("epoch = ", epoch)

    # for X, Y in tqdm(train_loader):
    correct = 0
    costs = 0
    model.train()
    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        H = model(X)    
        cost = criterion(H, Y)        

        # Multi-label
        # Y = torch.nn.functional.one_hot(Y, num_classes = 7)
        # cost = criterion(H, Y.float())        
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        # correct += sum(H.argmax(1) == Y).detach().cpu().numpy().tolist()
        # costs += cost.item()
    
    # print("train_acc = ", correct / len(train_y))
    model.eval()
    for X, Y in validation_loader:
        X = X.to(device)
        Y = Y.to(device)

        H = model(X)
        cost = criterion(H, Y)

        # Multi-label
        # Y = torch.nn.functional.one_hot(Y, num_classes = 7)
        # cost = criterion(H, Y.float())        
        
        # import pdb; pdb.set_trace();
        # correct += sum(H.argmax(1) == Y).detach().cpu().numpy().tolist()
        correct += ((H>0).int().flatten() == Y.flatten()).sum().detach().cpu().numpy().tolist()
        costs += cost.item()

    print("costs = ", costs / len(val_y))
    print("accs = ", correct / len(val_y) / n_classes)
        

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/3 [00:00<?, ?it/s]

epoch =  0


 33%|███▎      | 1/3 [06:04<12:08, 364.02s/it]

costs =  0.0022937423681704296
accs =  0.9754787160765945
epoch =  1


 67%|██████▋   | 2/3 [07:02<03:04, 184.49s/it]

costs =  0.0011422097095497206
accs =  0.9898057583689213
epoch =  2


100%|██████████| 3/3 [08:02<00:00, 160.71s/it]

costs =  0.0008142251327148665
accs =  0.9924231987877118


In [10]:
torch.save(model.state_dict(), "/content/model1.tar")

In [11]:
test_loader = DataLoader(MyDataset(tf, split='VALIDATION'),
                        batch_size=1,
                        shuffle=False,
                        drop_last=False,
                        pin_memory=True,
                        num_workers=4)

model.eval()
for i, (X, Y) in enumerate(test_loader):
    X = X.to(device)
    Y = Y.to(device)

    H = model(X)
    
    # print(i, '\t', end = ' ')
    
    # print((H.detach().cpu().numpy() > 0).astype(int))
    if (H.detach().cpu().numpy()> 0).sum() > 1:
        print(i, end = '\t')
        # print(H.detach().cpu().numpy())
        print(torch.sigmoid(H).detach().cpu().numpy())
        

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


864	[[1.0090820e-04 7.9357195e-01 6.2926245e-01 1.2621508e-03 6.4262364e-04
  3.6608524e-04 1.4074496e-03]]
1107	[[5.5234443e-04 6.4229244e-01 8.1986076e-01 4.4600642e-03 1.1619079e-03
  5.1426474e-04 1.2834900e-03]]
1333	[[0.02622994 0.0120701  0.01073295 0.01828396 0.5465905  0.77770764
  0.01628359]]
1338	[[0.02872238 0.04580449 0.0422614  0.1100883  0.7066227  0.65795493
  0.1086019 ]]
1427	[[0.00140179 0.00520676 0.00412202 0.8212284  0.00935387 0.0045406
  0.7952805 ]]
1428	[[0.00142637 0.02360287 0.00163478 0.87732536 0.00635521 0.00253769
  0.60007054]]
1431	[[0.00730347 0.03952754 0.01410131 0.6657243  0.06589578 0.02650563
  0.8979063 ]]
1433	[[0.00535882 0.01555057 0.00726261 0.71946496 0.03790953 0.03062603
  0.83920294]]
1440	[[0.00099336 0.00746942 0.00185136 0.8606901  0.00867432 0.00363262
  0.7197438 ]]
1444	[[0.00350348 0.01001075 0.04195823 0.6130612  0.02674309 0.02295746
  0.86305916]]
1447	[[0.00140725 0.02168655 0.01110754 0.6302884  0.01710393 0.01053271
  0.778

In [ ]:
H

tensor([[ 11.6344, -11.2087, -10.7682,  -9.6786,  -9.9556, -11.0021, -12.4964]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# efficientnet b0 multi-label
#   0%|          | 0/100 [00:00<?, ?it/s]epoch =  0
#   1%|          | 1/100 [01:10<1:56:12, 70.42s/it]costs =  0.0007745151134361509
# accs =  0.9922854387656702
# epoch =  1
#   2%|▏         | 2/100 [02:10<1:44:46, 64.15s/it]costs =  0.0006434683089703305
# accs =  0.9927675988428158
# epoch =  2
#   3%|▎         | 3/100 [03:10<1:40:47, 62.34s/it]costs =  0.0006036060756233277
# accs =  0.9927675988428158
# epoch =  3
#   4%|▍         | 4/100 [04:11<1:38:47, 61.75s/it]costs =  0.0005565035943934191
# accs =  0.9929053588648574
# epoch =  4
#   5%|▌         | 5/100 [05:11<1:37:05, 61.32s/it]costs =  0.0005646197611303379
# accs =  0.9919410387105662
# epoch =  5
#   6%|▌         | 6/100 [06:11<1:35:18, 60.83s/it]costs =  0.0005541736541314574
# accs =  0.9919410387105662
# epoch =  6
#   7%|▋         | 7/100 [07:11<1:33:52, 60.57s/it]costs =  0.000578838725959367
# accs =  0.9920099187215871
# epoch =  7
#   7%|▋         | 7/100 [07:52<1:44:32, 67.45s/it]

In [ ]:
# efficientnet b0 single-label
#   0%|          | 0/100 [00:00<?, ?it/s]epoch =  0
#   1%|          | 1/100 [01:21<2:14:48, 81.70s/it]costs =  0.0020630328236605788
# accs =  0.9810579969692795
# epoch =  1
#   2%|▏         | 2/100 [02:39<2:09:32, 79.31s/it]costs =  0.0011165186174783277
# accs =  0.9888414382146301
# epoch =  2
#   3%|▎         | 3/100 [03:56<2:06:44, 78.39s/it]costs =  0.0008010579946225467
# accs =  0.9915966386554622
# epoch =  3
#   4%|▍         | 4/100 [05:17<2:06:55, 79.33s/it]costs =  0.0006931836954488624
# accs =  0.9915966386554622
# epoch =  4
#   5%|▌         | 5/100 [06:35<2:04:37, 78.71s/it]costs =  0.0006687399283390072
# accs =  0.9922854387656702
# epoch =  5
#   5%|▌         | 5/100 [06:57<2:12:12, 83.50s/it]